In [92]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [8]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Cape Town,ZA,2020-04-18 18:49:04,-33.93,18.42,68.0,52,5,3.36,clear sky,0.0,0.0
1,1,Port Alfred,ZA,2020-04-18 18:53:17,-33.59,26.89,66.0,95,0,4.00,clear sky,0.0,0.0
2,2,Albany,US,2020-04-18 18:53:18,42.60,-73.97,43.0,69,98,1.01,light rain,0.6,0.0
3,3,Hilo,US,2020-04-18 18:53:18,19.73,-155.09,75.2,64,1,5.66,clear sky,0.0,0.0
4,4,Vaini,TO,2020-04-18 18:53:18,-21.20,-175.20,77.0,83,20,8.05,few clouds,0.0,0.0


In [9]:
city_data_df.dtypes

City_ID                       int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Weather Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [11]:
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

In [42]:
# Ask the customer to add a minimum temp, maximum temp, raining or snowing.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = input("Do you want it to be raining? (yes/no)")
snowing = input("Do you want it to be snowing? (yes/no)")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)no


In [43]:
raining

'no'

In [44]:
# Filer the dataset to find the cities that fit the criteria.
if raining == "no" and snowing == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                           & (city_data_df['Max Temp'] >= min_temp) &
                                           (city_data_df["Rain inches (last 3 hrs)"] == 0)
                                           & (city_data_df["Snow inches (last 3 hrs)"] == 0)]

elif raining == "no" and snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                           & (city_data_df['Max Temp'] >= min_temp) &
                                           (city_data_df["Rain inches (last 3 hrs)"] == 0)
                                           & (city_data_df["Snow inches (last 3 hrs)"] > 0)]

elif raining == "yes" and snowing == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                           & (city_data_df['Max Temp'] >= min_temp) &
                                           (city_data_df["Rain inches (last 3 hrs)"] > 0)
                                           & (city_data_df["Snow inches (last 3 hrs)"] == 0)]

else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                           & (city_data_df['Max Temp'] >= min_temp) &
                                           (city_data_df["Rain inches (last 3 hrs)"] > 0)
                                           & (city_data_df["Snow inches (last 3 hrs)"] > 0)]


In [45]:
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
3,3,Hilo,US,2020-04-18 18:53:18,19.73,-155.09,75.20,64,1,5.66,clear sky,0.0,0.0
4,4,Vaini,TO,2020-04-18 18:53:18,-21.20,-175.20,77.00,83,20,8.05,few clouds,0.0,0.0
10,10,Kahului,US,2020-04-18 18:49:54,20.89,-156.47,75.20,60,20,3.36,few clouds,0.0,0.0
11,11,Garowe,SO,2020-04-18 18:53:20,8.41,48.48,81.55,56,49,16.53,scattered clouds,0.0,0.0
12,12,Kapaa,US,2020-04-18 18:53:20,22.08,-159.32,77.00,73,20,12.75,few clouds,0.0,0.0


In [47]:
# Create a new DataFrame,
hotel_df = preferred_cities_df[['City', 'Country', 'Weather Description', 
                                'Max Temp', 'Lat', 'Lng']].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
3,Hilo,US,clear sky,75.20,19.73,-155.09,
4,Vaini,TO,few clouds,77.00,-21.20,-175.20,
10,Kahului,US,few clouds,75.20,20.89,-156.47,
11,Garowe,SO,scattered clouds,81.55,8.41,48.48,
12,Kapaa,US,few clouds,77.00,22.08,-159.32,


In [48]:
# Set parameters to search for a hotel.
params = {'radius': 5000,
         'type': 'lodging',
         'key': g_key}

In [50]:
## Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass
        
hotel_df.head(20)

,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
3,Hilo,US,clear sky,75.20,19.73,-155.09,Hilo Hawaiian Hotel
4,Vaini,TO,few clouds,77.00,-21.20,-175.20,Keleti Beach Resort
10,Kahului,US,few clouds,75.20,20.89,-156.47,Maui Seaside Hotel
11,Garowe,SO,scattered clouds,81.55,8.41,48.48,Curubo Hotel
12,Kapaa,US,few clouds,77.00,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [54]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = hotel_df[["Lat", "Lng"]]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
# Add markers
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [62]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp}</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [102]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.

fig



Figure(layout=FigureLayout(height='420px'))

In [64]:
# Create the output file (CSV).
output_data_file = "weather_data/Weather_vacation.csv"
# Export the City_data into CSV
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [69]:
dir(fig)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_default_layout',
 '_display_callbacks',
 '_dom_classes',
 '_errors_box',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_map',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_trait',
 '_property_lock',
 '_register_validator',
 '_remove_notifiers',
 '_